In [96]:
import pandas as pd # Fundamental para análise e manipulação de dados
import requests # Para fazer requisições HTTP (acessar APIs, baixar dados da web)
import os # Para interagir com o sistema operacional (manipular arquivos, pastas)
from datetime import datetime # Módulo para trabalhar com datas e horas
import yfinance as yt # Baixar dados do mercado financeiro (Yahoo Finance)
from pytrends.request import TrendReq # Extrair dados de tendências de busca (Google Trends)
from sidrapy import get_table # Função específica para buscar dados de tabelas do SIDRA (IBGE)
import math

In [97]:
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y/%m/%d')

data_raw_path = '../data_raw/'
os.makedirs(data_raw_path, exist_ok=True)

In [98]:
#Defini os indicadores a serem buscados
indicadores_bcb = {
    '432': 'selic_meta',      # SELIC (Meta) - Mensal
    '433': 'ipca_mensal',     # IPCA (Var. % mensal) - Mensal
    '24364': 'ibc_br',        # IBC-Br (Proxy do PIB, c/ ajuste) - Mensal
    '4393': 'icc_consumidor' # Índice de Confiança do Consumidor - Mensal
}

In [99]:
#Busca uma série temporal do SGS (BCB) e salva em um arquivo CSV na pasta data_raw

start_date_fmt = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
end_date_fmt = datetime.now().strftime('%d/%m/%Y')

def dados_bc (codigo_indicador, nome_indicador):

    url_api = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_indicador}/dados?formato=json&dataInicial={start_date_fmt}&dataFinal={end_date_fmt}"
    
    #Chamar a API
    response = requests.get(url_api)
    
    if response.status_code == 200: #Código 200 significa êxito
        dados_json = response.json()
        
        if not dados_json: # Checagem extra: e se a API retornar 'ok' mas sem dados?
            print(f"   > AVISO: Série {nome_indicador} não retornou dados.")
            return

        # Converte para DataFrame
        df = pd.DataFrame(dados_json)
        
        # Limpeza Tratamento de tipos
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df['valor'] = pd.to_numeric(df['valor'])
        
        # Renomeia a coluna 'valor' para o nome do indicador
        df = df.rename(columns={'valor': nome_indicador})
        
        # Salvar o arquivo CSV individual
        file_path = os.path.join(data_raw_path, f'{nome_indicador}.csv')
        df.to_csv(file_path, index=False)
        
        print(f"   > Sucesso. Salvo em: {file_path}")
        
    else:
        # (sua lógica de erro)
        print(f"   > ERRO! Falha ao buscar {nome_indicador}. Status: {response.status_code}")

In [100]:
for codigo, nome in indicadores_bcb.items():
    dados_bc(codigo, nome)

   > Sucesso. Salvo em: ../data_raw/selic_meta.csv
   > Sucesso. Salvo em: ../data_raw/ipca_mensal.csv
   > Sucesso. Salvo em: ../data_raw/ibc_br.csv
   > Sucesso. Salvo em: ../data_raw/icc_consumidor.csv


In [ ]:
# --- Bloco 6: Coleta de Dados do IBGE (SIDRA) (VERSÃO 6 - Robusta) ---

print("\n--- Iniciando Coleta de Dados do IBGE (SIDRA) ---")

try:
    # -----------------------------------------------------------
    # 1. Busca da Taxa de Desocupação (PNAD Contínua - TRIMESTRAL)
    # -----------------------------------------------------------
    
    desemprego_df = get_table(
        table_code='4095',
        territorial_level='1',
        ibge_territorial_code='all',
        variable='4099',
        period='all' 
    )
    
    # Limpeza Mínima (PNAD):
    if not desemprego_df.empty:
        desemprego_df = desemprego_df.loc[1:, ['V', 'D2C']]
        desemprego_df.columns = ['taxa_desocupacao', 'periodo_codigo_trimestre']
        
        # --- MUDANÇA: Adicionado errors='coerce' por segurança ---
        desemprego_df['taxa_desocupacao'] = pd.to_numeric(desemprego_df['taxa_desocupacao'], errors='coerce')
        
        # Filtro Local
        desemprego_df = desemprego_df[desemprego_df['periodo_codigo_trimestre'] >= '202001']
        
        file_path = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
        desemprego_df.to_csv(file_path, index=False)
        print(f"   > Sucesso (Trimestral). Salvo em: {file_path}")
    else:
        print("   > AVISO: Dados de desemprego (PNAD) não retornaram.")
        
    # -----------------------------------------------------------
    # 2. Busca do PIB (Contas Nacionais - TRIMESTRAL)
    # -----------------------------------------------------------
    
    pib_df = get_table(
        table_code='1620',
        territorial_level='1',
        ibge_territorial_code='all',
        variable='583',
        period='all'
    )
    
    # Limpeza Mínima (PIB):
    if not pib_df.empty:
        pib_df = pib_df.loc[1:, ['V', 'D2C']]
        pib_df.columns = ['pib_var_ano', 'periodo_codigo_trimestre']
        
        # --- MUDANÇA: Adicionado errors='coerce' para corrigir o bug do ".." ---
        pib_df['pib_var_ano'] = pd.to_numeric(pib_df['pib_var_ano'], errors='coerce')
        
        # Filtro Local
        pib_df = pib_df[pib_df['periodo_codigo_trimestre'] >= '202001']
        
        file_path = os.path.join(data_raw_path, 'pib_trimestral.csv')
        pib_df.to_csv(file_path, index=False)
        print(f"   > Sucesso (Trimestral). Salvo em: {file_path}")
    else:
        print("   > AVISO: Dados do PIB (Contas Nacionais) não retornaram.")
        
except Exception as e:
    print(f"   > ERRO GERAL! Falha ao buscar dados do IBGE: {e}")

print("\n--- Coleta de Dados do IBGE Finalizada! ---")


--- Iniciando Coleta de Dados do IBGE (SIDRA) ---
   > Buscando: Taxa de Desocupação (PNAD Tabela 4095)...
   > Período usado: 'all' (vamos filtrar localmente)
   > Sucesso (Trimestral). Salvo em: ../data_raw/desemprego_trimestral.csv
   > Buscando: PIB Variação Anual (Contas Trimestrais Tabela 1620)...
   > Período usado: 'all' (vamos filtrar localmente)
   > Sucesso (Trimestral). Salvo em: ../data_raw/pib_trimestral.csv

--- Coleta de Dados do IBGE Finalizada! ---
